Grant application $\Leftrightarrow$ Output publications $\Leftrightarrow$ Cited papers

In [16]:
import json
import pandas as pd
import requests
from time import time

pd.set_option('display.precision', 3)

In [17]:
# load the data
grants = pd.read_csv("data/GrantWithAbstracts.csv", delimiter=";")
publications = pd.read_csv("data/OutputdataScientificPublication.csv", delimiter=";")

In [18]:
# peek at grants
display(grants)
grants.columns.to_list()

,GrantNumber,GrantNumberString,Title,TitleEnglish,ResponsibleApplicantName,FundingInstrumentPublished,FundingInstrumentReporting,FundingInstrumentLevel1,Institute,InstituteCity,...,LaySummaryLead_De,LaySummary_De,LaySummaryLead_Fr,LaySummary_Fr,LaySummaryLead_It,LaySummary_It,State,CallFullTitle,CallEndDate,CallDecisionYear
0,154855,PZ00P2_154855,Integrating Dynamic In Vivo Imaging And Comput...,NaN,"Aiyangar, Ameet",Ambizione,Ambizione,Careers,Empa,8600 Dübendorf,...,Kreuzschmerzen können aus individueller Sicht...,<p>Obwohl Schmerzen im unteren R&uuml;cken ein...,NaN,NaN,NaN,NaN,Completed,Ambizione 2014,2014-02-14T00:00:00Z,2014
1,151068,10CO16_151068,1. Berner Arbeitstreffen zur Visuellen Rhetorik,NaN,"Scheuermann, Arne",Scientific Conferences,Scientific Conferences,Science communication,Forschungsschwerpunkt Kommunikationsdesign Y (...,3027 Bern,...,NaN,NaN,NaN,NaN,NaN,NaN,Completed,NaN,NaN,2013
2,180481,B-0010_180481,Kunst als Entscheidung – Performancekunst der ...,NaN,"Renz, Seraina",Publication grants,Publication grants,Science communication,Kunsthistorisches Institut Universität Zürich,8006 Zürich,...,NaN,NaN,NaN,NaN,NaN,NaN,Completed,Publication grants 2017,2018-01-01T00:00:00Z,2017
3,34238,4031-034238,Nowcasting von Niederschlägen mittels Radar fü...,NaN,"Joss, Jürg",NRP 31 Climatic Changes and Natural Hazards,National Research Programmes (NRPs),Programmes,Osservatorio Ticinese di SMA,6605 Locarno,...,NaN,NaN,NaN,NaN,NaN,NaN,Completed,NaN,NaN,1992
4,225667,P5R5PH_225667,The Role of Art Schools in the Construction of...,NaN,"Rérat, Melissa",Return CH Postdoc.Mobility,Fellowships,Careers,eikones – Zentrum für die Theorie und Geschich...,4051 Basel,...,"Während die Beteiligung von Universitäten, Mus...",Das Projekt konzentriert sich auf die Jahre 19...,NaN,NaN,NaN,NaN,Ongoing,Return CH Postdoc.Mobility 2024 February,2024-02-01T00:00:00Z,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88356,209488,IZINZ1_209488,Homophily and the Spread of (Dis)Information i...,Homophily and the Spread of (Dis)Information i...,"Brandes, Ulrik",Bilateral programmes,Bilateral programmes,Programmes,Professur für Soziale Netzwerke D-GESS ETH Zürich,8092 Zürich,...,Soziale Muster können von Unternehmen ausgenut...,Ein h&auml;ufig zu beobachtendes soziales Must...,NaN,NaN,NaN,NaN,Ongoing,Trilateral Call with India - SNSF-ICSSR-MoES 2021,2021-11-30T00:00:00Z,2022
88357,159260,100017_159260,Gesellschaftliche Positionierung von jungen Er...,The societal positioning of young second-gener...,"Mey, Eva",Project funding,Project funding,Projects,Institut für Delinquenz u. Kriminalpräventio D...,8037 Zürich,...,Im Übergang ins Erwachsenenalter beginnen jung...,<p><strong>Inhalt und Ziel des Forschungsproje...,NaN,NaN,NaN,NaN,Completed,Project funding 2014 (01.10.2014),2014-10-01T00:00:00Z,2015
88358,5662,3100-005662,Untersuchungen über den Protein- und Nucleinsä...,NaN,"Chen, Pei-Shen",Project funding,Project funding,Projects,Zoologisches Institut Universität Zürich-Irchel,8057 Zürich,...,NaN,NaN,NaN,NaN,NaN,NaN,Completed,NaN,NaN,1975
88359,42382,2100-042382,Renovation du diffractomètre des RX,NaN,"Fontboté, Lluis",Project funding,Project funding,Projects,Département des sciences de la Terre Universit...,1205 Genève,...,NaN,NaN,NaN,NaN,NaN,NaN,Completed,NaN,NaN,1995


['GrantNumber',
 'GrantNumberString',
 'Title',
 'TitleEnglish',
 'ResponsibleApplicantName',
 'FundingInstrumentPublished',
 'FundingInstrumentReporting',
 'FundingInstrumentLevel1',
 'Institute',
 'InstituteCity',
 'InstituteCountry',
 'ResearchInstitution',
 'ResearchInstitutionType',
 'MainDisciplineNumber',
 'MainDiscipline',
 'MainDiscipline_Level1',
 'MainDiscipline_Level2',
 'AllDisciplines',
 'MainFieldOfResearchNumber',
 'MainFieldOfResearch',
 'MainFieldOfResearch_LevelA',
 'MainFieldOfResearch_LevelB',
 'AllFieldOfResearchs',
 'EffectiveGrantStartDate',
 'EffectiveGrantEndDate',
 'AmountGrantedAllSets',
 'Keywords',
 'Abstract',
 'LaySummaryLead_En',
 'LaySummary_En',
 'LaySummaryLead_De',
 'LaySummary_De',
 'LaySummaryLead_Fr',
 'LaySummary_Fr',
 'LaySummaryLead_It',
 'LaySummary_It',
 'State',
 'CallFullTitle',
 'CallEndDate',
 'CallDecisionYear']

In [19]:
# percentage nontrivial values per column in grants (only those with trivial values are shown)
temp = (~grants.isna()).sum() / len(grants) * 100
temp[temp < 99]

TitleEnglish         43.804
Institute            92.342
InstituteCity        91.758
InstituteCountry     91.981
Keywords             70.297
Abstract             48.287
LaySummaryLead_En     2.935
LaySummary_En        14.528
LaySummaryLead_De    17.546
LaySummary_De        19.931
LaySummaryLead_Fr    10.511
LaySummary_Fr        11.484
LaySummaryLead_It     2.373
LaySummary_It         2.391
CallFullTitle        52.760
CallEndDate          52.760
dtype: float64

In [20]:
# peek at publications
display(publications)
publications.columns.to_list()

,ScientificPublicationId,GrantNumber,ScientificPublication_PeerReviewStatus,ScientificPublication_Type,ScientificPublication_Title,ScientificPublication_Author,ScientificPublication_State,ScientificPublication_Year,ScientificPublication_ISBN,ScientificPublication_DOI,...,ScientificPublication_Url,ScientificPublication_PublicationTitle,ScientificPublication_Publisher,ScientificPublication_Editor,ScientificPublication_Volume,ScientificPublication_IssueNumber,ScientificPublication_FirstPageNumber,ScientificPublication_LastPageNumber,ScientificPublication_ProceedingLocation,ScientificPublication_Abstract
0,4CEC2482-3B76-4204-9CDC-206E25E55DEE,198227,Peer-reviewed,Original contribution to science journal,"Forest-floor respiration, N2O fluxes, and CH4 ...","Krebs, Luana; Burri, Susanne; Feigenwinter, Ir...",Published,2024.0,NaN,10.5194/bg-21-2005-2024,...,https://doi.org/10.5194/bg-21-2005-2024,Biogeosciences,NaN,NaN,21,8,2005,2028,NaN,Abstract. Forest ecosystems play an important ...
1,5D76C82E-7792-4DBE-AF23-00386D0751E0,198227,Peer-reviewed,Original contribution to science journal,Disentangling effects of natural and anthropog...,"Wang, You-Ren; Buchmann, Nina; Hessen, Dag O.;...",Published,2022.0,NaN,10.1016/j.scitotenv.2022.156326,...,http://doi.org/10.1016/j.scitotenv.2022.156326,Science of The Total Environment,NaN,NaN,839,NaN,156326,156326,NaN,NaN
2,D4870A4A-E7B6-4195-A565-A31AA53F9D07,152599,Not peer-reviewed,Review article,Miasto dobrze wymieszane (The well mixed city),"Klumpner, Hubert",Published,2017.0,NaN,NaN,...,NaN,Architektura Murator,NaN,NaN,10,277,32,38,NaN,NaN
3,FC7E2263-17FD-4DB1-8CF2-1939CD791F42,198227,Peer-reviewed,Original contribution to science journal,Monitoring of carbon-water fluxes at Eurasian ...,"Xie, Mingjuan; Ma, Xiaofei; Wang, Yuangang; Li...",Published,2023.0,NaN,10.1038/s41597-023-02473-9,...,http://doi.org/10.1038/s41597-023-02473-9,Scientific Data,NaN,NaN,10,1,587,587,NaN,Abstract Simulating the carbon-water fluxes at...
4,2CB6405E-BEC9-47EC-A91B-15FBF22155C6,198227,Peer-reviewed,Original contribution to science journal,Leaf-level coordination principles propagate t...,"Gomarasca, Ulisse; Migliavacca, Mirco; Kattge,...",Published,2023.0,NaN,10.1038/s41467-023-39572-5,...,http://doi.org/10.1038/s41467-023-39572-5,Nature Communications,NaN,NaN,14,1,3948,3948,NaN,AbstractFundamental axes of variation in plant...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204245,FDA79332-11F0-41E1-9047-2E27988B820E,174293,Peer-reviewed,Review article,Charge migration and charge transfer in molecu...,"Wörner, Hans Jakob; Arrell, Christopher A.; Ba...",Published,2017.0,NaN,10.1063/1.4996505,...,NaN,Structural Dynamics,NaN,NaN,4,6,061508,061508,NaN,NaN
204246,E7AA3477-CDA2-41C2-ABCD-2659910F737C,178753,Peer-reviewed,Original contribution to science journal,Vegetation type is an important predictor of t...,"Oehri, Jacqueline; Schaepman-Strub, Gabriela; ...",Published,2022.0,NaN,10.1038/s41467-022-34049-3,...,http://doi.org/10.1038/s41467-022-34049-3,Nature Communications,NaN,NaN,13,1,6379,6379,NaN,Abstract Despite the importance of high-latitu...
204247,5269D231-12EA-4A82-803B-06B04F623AFE,147630,Not peer-reviewed,Other literature,Molecular ecology studies of species radiation...,"de la Harpe, Marylaure; Paris, Margot; Karger,...",Published,2017.0,NaN,10.1111/mec.14110,...,NaN,Molecular Ecology,NaN,NaN,26,10,2608,2622,NaN,NaN
204248,E0ADD3BB-6BF2-46FD-8EA4-179229DE078A,166107,Peer-reviewed,Proceedings,Active Learning and Proofreading for Delineati...,"Mosinska, Agata; Tarnawski, Jakub ; Fua, Pascal",Published,2017.0,NaN,NaN,...,https://infoscience.epfl.ch/record/229472,Medical Image Computing and Computer-Assisted...,"Quebec City, Canada: Springer",NaN,NaN,NaN,NaN,NaN,NaN,NaN


['ScientificPublicationId',
 'GrantNumber',
 'ScientificPublication_PeerReviewStatus',
 'ScientificPublication_Type',
 'ScientificPublication_Title',
 'ScientificPublication_Author',
 'ScientificPublication_State',
 'ScientificPublication_Year',
 'ScientificPublication_ISBN',
 'ScientificPublication_DOI',
 'ScientificPublication_ImportSource',
 'ScientificPublication_OpenAccessStatusYesNo',
 'ScientificPublication_OpenAccessStatus',
 'ScientificPublication_Url',
 'ScientificPublication_PublicationTitle',
 'ScientificPublication_Publisher',
 'ScientificPublication_Editor',
 'ScientificPublication_Volume',
 'ScientificPublication_IssueNumber',
 'ScientificPublication_FirstPageNumber',
 'ScientificPublication_LastPageNumber',
 'ScientificPublication_ProceedingLocation',
 'ScientificPublication_Abstract']

In [21]:
# percentage nontrivial values per column in publications (only those with trivial values are shown)
temp = (~publications.isna()).sum() / len(publications) * 100
temp[temp < 99]

ScientificPublication_Author                98.212
ScientificPublication_Year                  91.539
ScientificPublication_ISBN                  11.766
ScientificPublication_DOI                   66.580
ScientificPublication_ImportSource          98.749
ScientificPublication_OpenAccessStatus      74.388
ScientificPublication_Url                   47.926
ScientificPublication_PublicationTitle      92.740
ScientificPublication_Publisher             21.962
ScientificPublication_Editor                 9.819
ScientificPublication_Volume                66.713
ScientificPublication_IssueNumber           47.307
ScientificPublication_FirstPageNumber       81.174
ScientificPublication_LastPageNumber        69.027
ScientificPublication_ProceedingLocation     3.979
ScientificPublication_Abstract              53.106
dtype: float64

many publications that supposedly have no abstract do, in fact, have abstracts according to S2AG. examples:

- title="Active Learning and Proofreading for Delineation of Curvilinear Structures"

this may be due to the publication type. we should be able to use the Semantic Scholar API to retrieve the missing information.

18 publications have grant numbers that do not appear in the grants dataset

- affected grant numbers: {227719, 227437, 231184, 227570, 222740, 227797, 231162, 228381, 227774, 227871}
- these pubs are quite new (>=2024), so maybe grants dataset is outdated?
- for now we just remove them

In [22]:
# removing publications without valid grant ids
valid_grant_numbers = set(grants.GrantNumber.tolist())
publications = publications[publications.GrantNumber.apply(lambda x: x in valid_grant_numbers)]
publications

,ScientificPublicationId,GrantNumber,ScientificPublication_PeerReviewStatus,ScientificPublication_Type,ScientificPublication_Title,ScientificPublication_Author,ScientificPublication_State,ScientificPublication_Year,ScientificPublication_ISBN,ScientificPublication_DOI,...,ScientificPublication_Url,ScientificPublication_PublicationTitle,ScientificPublication_Publisher,ScientificPublication_Editor,ScientificPublication_Volume,ScientificPublication_IssueNumber,ScientificPublication_FirstPageNumber,ScientificPublication_LastPageNumber,ScientificPublication_ProceedingLocation,ScientificPublication_Abstract
0,4CEC2482-3B76-4204-9CDC-206E25E55DEE,198227,Peer-reviewed,Original contribution to science journal,"Forest-floor respiration, N2O fluxes, and CH4 ...","Krebs, Luana; Burri, Susanne; Feigenwinter, Ir...",Published,2024.0,NaN,10.5194/bg-21-2005-2024,...,https://doi.org/10.5194/bg-21-2005-2024,Biogeosciences,NaN,NaN,21,8,2005,2028,NaN,Abstract. Forest ecosystems play an important ...
1,5D76C82E-7792-4DBE-AF23-00386D0751E0,198227,Peer-reviewed,Original contribution to science journal,Disentangling effects of natural and anthropog...,"Wang, You-Ren; Buchmann, Nina; Hessen, Dag O.;...",Published,2022.0,NaN,10.1016/j.scitotenv.2022.156326,...,http://doi.org/10.1016/j.scitotenv.2022.156326,Science of The Total Environment,NaN,NaN,839,NaN,156326,156326,NaN,NaN
2,D4870A4A-E7B6-4195-A565-A31AA53F9D07,152599,Not peer-reviewed,Review article,Miasto dobrze wymieszane (The well mixed city),"Klumpner, Hubert",Published,2017.0,NaN,NaN,...,NaN,Architektura Murator,NaN,NaN,10,277,32,38,NaN,NaN
3,FC7E2263-17FD-4DB1-8CF2-1939CD791F42,198227,Peer-reviewed,Original contribution to science journal,Monitoring of carbon-water fluxes at Eurasian ...,"Xie, Mingjuan; Ma, Xiaofei; Wang, Yuangang; Li...",Published,2023.0,NaN,10.1038/s41597-023-02473-9,...,http://doi.org/10.1038/s41597-023-02473-9,Scientific Data,NaN,NaN,10,1,587,587,NaN,Abstract Simulating the carbon-water fluxes at...
4,2CB6405E-BEC9-47EC-A91B-15FBF22155C6,198227,Peer-reviewed,Original contribution to science journal,Leaf-level coordination principles propagate t...,"Gomarasca, Ulisse; Migliavacca, Mirco; Kattge,...",Published,2023.0,NaN,10.1038/s41467-023-39572-5,...,http://doi.org/10.1038/s41467-023-39572-5,Nature Communications,NaN,NaN,14,1,3948,3948,NaN,AbstractFundamental axes of variation in plant...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204245,FDA79332-11F0-41E1-9047-2E27988B820E,174293,Peer-reviewed,Review article,Charge migration and charge transfer in molecu...,"Wörner, Hans Jakob; Arrell, Christopher A.; Ba...",Published,2017.0,NaN,10.1063/1.4996505,...,NaN,Structural Dynamics,NaN,NaN,4,6,061508,061508,NaN,NaN
204246,E7AA3477-CDA2-41C2-ABCD-2659910F737C,178753,Peer-reviewed,Original contribution to science journal,Vegetation type is an important predictor of t...,"Oehri, Jacqueline; Schaepman-Strub, Gabriela; ...",Published,2022.0,NaN,10.1038/s41467-022-34049-3,...,http://doi.org/10.1038/s41467-022-34049-3,Nature Communications,NaN,NaN,13,1,6379,6379,NaN,Abstract Despite the importance of high-latitu...
204247,5269D231-12EA-4A82-803B-06B04F623AFE,147630,Not peer-reviewed,Other literature,Molecular ecology studies of species radiation...,"de la Harpe, Marylaure; Paris, Margot; Karger,...",Published,2017.0,NaN,10.1111/mec.14110,...,NaN,Molecular Ecology,NaN,NaN,26,10,2608,2622,NaN,NaN
204248,E0ADD3BB-6BF2-46FD-8EA4-179229DE078A,166107,Peer-reviewed,Proceedings,Active Learning and Proofreading for Delineati...,"Mosinska, Agata; Tarnawski, Jakub ; Fua, Pascal",Published,2017.0,NaN,NaN,...,https://infoscience.epfl.ch/record/229472,Medical Image Computing and Computer-Assisted...,"Quebec City, Canada: Springer",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
grant_number2grant = dict(zip(
    grants.GrantNumber, grants.to_dict(orient="records")
))

grant_number2publication_ids = {
    grant_number: publications.ScientificPublicationId[row_idxs].tolist()
    for grant_number, row_idxs in publications.groupby("GrantNumber").groups.items()
}
publication_id2publication = dict(zip(
    publications.ScientificPublicationId, publications.to_dict(orient="records")
))

In [28]:
# case study with richard's grants and papers
hahnloser_grant_numbers = set(
    grants[grants.ResponsibleApplicantName.apply(lambda x: "Hahnloser" in x)].GrantNumber.tolist()
)
for grant_number in hahnloser_grant_numbers:
    grant = grant_number2grant[grant_number]
    publication_ids = grant_number2publication_ids.get(grant_number, [])

    print(f"Grant {grant['GrantNumber']}")
    print(grant["Title"])
    print(grant["Abstract"])
    print()

    for publication_id in publication_ids:
        publication = publication_id2publication[publication_id]
        
        print(f"Publication {publication['ScientificPublicationId']}")
        print(publication["ScientificPublication_Title"])
        print(publication["ScientificPublication_Abstract"])
        print()

    print("-" * 100)

Grant 130470
Neural Circuit Reconstruction
The history of natural sciences has taught us that a new discipline emerges whenever the length scale of investigation is reduced by about three orders of magnitude.  The recent development of 3-D electron microscopy (3DEM) techniques induces such a transition by allowing the ultrastructural analysis of biological tissues throughout large volumes (up to a few hundred micrometers in each dimension). Although 3DEM has an enormous potential for advancing our understanding of brain function in health and disease, its biomedical application is currently impeded by the lack of accessory technologies, particularly informatics tools for the automated analysis of large and complex sets of image data. The scope of our Sinergia Proposal is to develop technologies for using 3DEM and reconstruct neuronal circuits in the vertebrate brain.High-throughput 3DEM hardware for large-scale brain imaging is currently developed at a rapid pace. However, there are ma

In [10]:
# Obtain the papers cited by each publication
for grant_number in hahnloser_grant_numbers:
    grant = grant_number2grant[grant_number]
    publication_ids = grant_number2publication_ids.get(grant_number, [])
    publications = [
        publication_id2publication.get(_, {}) for _ in publication_ids
    ]

    titles = [_.get("ScientificPublication_Title", "") for _ in publications]
    response = requests.post(
        "http://zulit.ini.uzh.ch/ml-api/title-generic-search/v1.0",
        data = json.dumps({"titles": titles}),
        headers = {"Content-Type": "application/json"},
    ).json()["response"]
    
    for publication, _response in zip(publications, response):
        publication["corpus_id"] = _response.get("id_int")

In [11]:
for grant_number in hahnloser_grant_numbers:
    grant = grant_number2grant[grant_number]
    publication_ids = grant_number2publication_ids.get(grant_number, [])
    publications = [
        publication_id2publication.get(_, {}) for _ in publication_ids
    ]
    print(len(publications), sum(bool(_["corpus_id"]) for _ in publications))

10 1
0 0
0 0
0 0
12 9
7 3
9 6
0 0
